In [2]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np

os.chdir(r"C:\Users\iyedm\OneDrive\Desktop\facify")
print(os.getcwd())


C:\Users\iyedm\OneDrive\Desktop\facify


In [3]:
import torch

# Check PyTorch version
print("PyTorch version:", torch.__version__)

# Check if CUDA is available
print("CUDA available:", torch.cuda.is_available())

# Show CUDA device count
print("Number of CUDA devices:", torch.cuda.device_count())

# Show the name of the current CUDA device
if torch.cuda.is_available():
    print("Current CUDA device:", torch.cuda.get_device_name(torch.cuda.current_device()))


PyTorch version: 2.9.0+cu128
CUDA available: True
Number of CUDA devices: 1
Current CUDA device: NVIDIA GeForce RTX 4060 Laptop GPU


In [ ]:
from backend.models.stylegan3_conditional import ConditionalStyleGAN2Generator
from backend.models.stylegan3_conditional_discriminator import ConditionalDiscriminator

# ----------------------------
# Paths
# ----------------------------
RAW_IMAGES_DIR = "data/processed/faces"           # preprocessed 128x128 images
EMBEDDINGS_PATH = "data/embeddings/faces.npy"    # embeddings
PRETRAINED_PKL = "backend/models/model_weights/stylegan2-ffhq-512x512.pkl"
SAVE_WEIGHTS = "backend/models/model_weights/stylegan2_conditional.pth"

# ----------------------------
# Training hyperparameters
# ----------------------------
BATCH_SIZE = 4
LR = 1e-4
EPOCHS = 5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"[INFO] Using device: {DEVICE}")


[INFO] Using device: cuda


In [5]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import numpy as np

class FacesDataset(Dataset):
    """
    Dataset that yields (image_tensor, embedding_tensor) pairs.
    Expects embeddings saved as: np.save(path, {"embeddings": embeddings_array, "filenames": filenames})
    - embeddings_array shape: (N, 512)
    - filenames: list of filenames corresponding to rows of embeddings_array
    """
    def __init__(self, images_dir, embeddings_path, transform=None):
        self.images_dir = images_dir
        self.transform = transform

        # Load the .npy which contains a dict
        print(f"[DEBUG] Loading embeddings from: {embeddings_path}")
        loaded = np.load(embeddings_path, allow_pickle=True)
        # np.load may return an array-like with a dict inside; handle both possibilities
        if isinstance(loaded, np.ndarray) and loaded.dtype == object:
            data = loaded.item()
        elif isinstance(loaded, dict):
            data = loaded
        else:
            # If someone saved plain array (rare), assume it's embeddings without filenames
            raise ValueError("[ERROR] Unexpected embeddings file format. Expected dict with 'embeddings' and 'filenames'.")

        if "embeddings" not in data or "filenames" not in data:
            raise ValueError("[ERROR] Embeddings file must contain keys 'embeddings' and 'filenames'.")

        embeddings_array = np.asarray(data["embeddings"])
        filenames_list = list(data["filenames"])

        print(f"[DEBUG] Embeddings loaded: {embeddings_array.shape[0]} items, embeddings dim = {embeddings_array.shape[1] if embeddings_array.ndim>1 else 'unknown'}")

        # Build mapping filename -> embedding (filenames in embeddings may include paths or just names)
        self.embedding_map = {}
        for fname, emb in zip(filenames_list, embeddings_array):
            # ensure the filename is exactly the basename
            base = os.path.basename(fname)
            self.embedding_map[base] = emb

        # Now scan images_dir and keep only files that have embeddings
        all_image_files = sorted([f for f in os.listdir(images_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))])
        kept = []
        skipped_no_embedding = []
        for f in all_image_files:
            if f in self.embedding_map:
                kept.append(f)
            else:
                skipped_no_embedding.append(f)

        if len(skipped_no_embedding) > 0:
            print(f"[WARN] {len(skipped_no_embedding)} images in '{images_dir}' have NO matching embedding and will be skipped (examples): {skipped_no_embedding[:5]}")

        if len(kept) == 0:
            raise RuntimeError(f"[ERROR] No images in '{images_dir}' matched embeddings from '{embeddings_path}'.")

        self.filenames = kept
        # create a parallel list of embeddings in the same order as self.filenames
        self.embeddings = [self.embedding_map[f] for f in self.filenames]

        print(f"[INFO] Using {len(self.filenames)} image-embedding pairs for training.")

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.filenames[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform is not None:
            image = self.transform(image)
        # embeddings stored as numpy arrays; convert to torch tensor
        embedding = torch.tensor(self.embeddings[idx], dtype=torch.float)
        return image, embedding


In [6]:
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

dataset = FacesDataset("data/processed/faces", "data/embeddings/faces.npy", transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2, pin_memory=True)

print(f"[INFO] Dataloader ready. {len(dataset)} samples, batch_size={dataloader.batch_size}")


[DEBUG] Loading embeddings from: data/embeddings/faces.npy
[DEBUG] Embeddings loaded: 7136 items, embeddings dim = 512
[WARN] 83 images in 'data/processed/faces' have NO matching embedding and will be skipped (examples): ['1 (1047).jpg', '1 (120).jpg', '1 (1323).jpg', '1 (1519).jpg', '1 (1568).jpg']
[INFO] Using 7136 image-embedding pairs for training.
[INFO] Dataloader ready. 7136 samples, batch_size=4


In [7]:
# backend/scripts/debug_dataset.py
import os
import torch
from torchvision.utils import make_grid, save_image
from torchvision import transforms
from torch.utils.data import DataLoader

# Import the fixed dataset class (adjust path if you placed it elsewhere)

DEBUG_OUT_DIR = "debug_samples"
os.makedirs(DEBUG_OUT_DIR, exist_ok=True)

# same transform used for training (images normalized to [-1,1])
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

print("[DEBUG] Creating dataset + dataloader...")
dataset = FacesDataset("data/processed/faces", "data/embeddings/faces.npy", transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=False)

# get a small batch
images, embeddings = next(iter(dataloader))

# denormalize images for saving/viewing
images_denorm = images * 0.5 + 0.5  # from [-1,1] -> [0,1]
grid = make_grid(images_denorm[:3], nrow=3)  # first 3 images

save_path = os.path.join(DEBUG_OUT_DIR, "sample_grid.png")
save_image(grid, save_path)
print(f"[INFO] Saved sample image grid to: {save_path}")

# print details for first 3 samples
print("\n[INFO] First 3 samples details:")
for i in range(min(3, images.size(0))):
    # filename
    fname = dataset.filenames[i]
    print(f"  {i+1}) filename: {fname}")
    print(f"     image tensor shape: {tuple(images[i].shape)}")
    print(f"     embedding shape: {tuple(embeddings[i].shape)}")
    norm = torch.norm(embeddings[i]).item()
    print(f"     embedding L2 norm: {norm:.4f}")

# Quick check: ensure number of images == number of embeddings used
print(f"\n[INFO] Dataset length: {len(dataset)} (should be 7136)")


[DEBUG] Creating dataset + dataloader...
[DEBUG] Loading embeddings from: data/embeddings/faces.npy
[DEBUG] Embeddings loaded: 7136 items, embeddings dim = 512
[WARN] 83 images in 'data/processed/faces' have NO matching embedding and will be skipped (examples): ['1 (1047).jpg', '1 (120).jpg', '1 (1323).jpg', '1 (1519).jpg', '1 (1568).jpg']
[INFO] Using 7136 image-embedding pairs for training.
[INFO] Saved sample image grid to: debug_samples\sample_grid.png

[INFO] First 3 samples details:
  1) filename: 1 (1).jpeg
     image tensor shape: (3, 128, 128)
     embedding shape: (512,)
     embedding L2 norm: 1.0000
  2) filename: 1 (1).jpg
     image tensor shape: (3, 128, 128)
     embedding shape: (512,)
     embedding L2 norm: 1.0000
  3) filename: 1 (1).png
     image tensor shape: (3, 128, 128)
     embedding shape: (512,)
     embedding L2 norm: 1.0000

[INFO] Dataset length: 7136 (should be 7136)


In [ ]:
# backend/scripts/train_dry_run_gpu.py
import os
import pickle
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn

# Import your dataset and models (adjust paths if needed)
from backend.models.stylegan3_conditional import ConditionalStyleGAN2Generator
from backend.models.stylegan3_conditional_discriminator import ConditionalDiscriminator

# -------------------
# GPU setup
# -------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[INFO] Using device: {DEVICE}")
torch.backends.cudnn.benchmark = True  # optimize conv layers for GPU

# -------------------
# Config
# -------------------
IMAGES_DIR = "data/processed/faces"
EMBEDDINGS_PATH = "data/embeddings/faces.npy"
PRETRAINED_PKL = "backend/models/model_weights/stylegan2-ffhq-512x512.pkl"
BATCH_SIZE = 4
NUM_BATCHES = 2  # dry-run
LR = 1e-4

# -------------------
# Dataset + DataLoader
# -------------------
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

dataset = FacesDataset(IMAGES_DIR, EMBEDDINGS_PATH, transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True,
                        num_workers=4, pin_memory=True)
print(f"[INFO] Dataloader ready: {len(dataset)} samples, batch_size={BATCH_SIZE}")

# -------------------
# Models
# -------------------
gen = ConditionalStyleGAN2Generator(latent_dim=512, embed_dim=512, img_channels=3).to(DEVICE)
disc = ConditionalDiscriminator(img_channels=3, embed_dim=512).to(DEVICE)
print("[INFO] Generator and Discriminator instantiated on GPU.")

# -------------------
# Load pretrained weights (best-effort)
# -------------------
if os.path.exists(PRETRAINED_PKL):
    try:
        print(f"[INFO] Loading pretrained .pkl from: {PRETRAINED_PKL}")
        with open(PRETRAINED_PKL, "rb") as f:
            ckpt = pickle.load(f)
        candidate_keys = list(ckpt.keys())
        print(f"[DEBUG] Keys in .pkl: {candidate_keys[:10]} ...")
        g_state = None
        for k in ("G_ema", "G", "g_ema", "g"):
            if k in ckpt:
                print(f"[INFO] Found generator key: {k}")
                if isinstance(ckpt[k], dict) and all(isinstance(v, (torch.Tensor, type(None))) for v in ckpt[k].values()):
                    g_state = ckpt[k]
                elif hasattr(ckpt[k], "state_dict"):
                    g_state = ckpt[k].state_dict()
                break
        if g_state:
            res = gen.load_state_dict(g_state, strict=False)
            try:
                missing = res.missing_keys
                unexpected = res.unexpected_keys
            except Exception:
                missing = res.get("missing_keys", [])
                unexpected = res.get("unexpected_keys", [])
            print(f"[INFO] load_state_dict done. missing_keys={len(missing)}, unexpected_keys={len(unexpected)}")
            if missing: print(f"[DEBUG] missing keys: {missing[:10]}")
            if unexpected: print(f"[DEBUG] unexpected keys: {unexpected[:10]}")
        else:
            print("[WARN] No usable generator state_dict found; proceeding without pretrained weights.")
    except Exception as e:
        print(f"[ERROR] Exception loading .pkl: {e}. Continuing without weights.")
else:
    print("[WARN] Pretrained .pkl not found; training from scratch.")

# -------------------
# Optimizers & Loss
# -------------------
opt_G = torch.optim.Adam(gen.parameters(), lr=LR, betas=(0.0, 0.99))
opt_D = torch.optim.Adam(disc.parameters(), lr=LR, betas=(0.0, 0.99))
criterion = nn.BCEWithLogitsLoss()

# -------------------
# Dry-run training loop
# -------------------
print("[INFO] Starting dry-run training on GPU...")
for batch_idx, (images, embeddings) in enumerate(dataloader, 1):
    images = images.to(DEVICE, non_blocking=True)
    embeddings = embeddings.to(DEVICE, non_blocking=True)

    # ----- Discriminator step -----
    opt_D.zero_grad()
    with torch.no_grad():
        fake = gen(embeddings)
    # Upsample if necessary
    if fake.shape[2:] != images.shape[2:]:
        fake = F.interpolate(fake, size=images.shape[2:], mode="bilinear", align_corners=False)
        print(f"[DEBUG] Upsampled fake to {tuple(fake.shape)}")

    real_labels = torch.ones(images.size(0), 1, device=DEVICE)
    fake_labels = torch.zeros(images.size(0), 1, device=DEVICE)

    real_out = disc(images, embeddings)
    fake_out = disc(fake.detach(), embeddings)
    loss_D = 0.5 * (criterion(real_out, real_labels) + criterion(fake_out, fake_labels))
    loss_D.backward()
    opt_D.step()

    # ----- Generator step -----
    opt_G.zero_grad()
    fake = gen(embeddings)
    if fake.shape[2:] != images.shape[2:]:
        fake = F.interpolate(fake, size=images.shape[2:], mode="bilinear", align_corners=False)
    fake_out = disc(fake, embeddings)
    loss_G = criterion(fake_out, real_labels)
    loss_G.backward()
    opt_G.step()

    print(f"[DEBUG] Batch {batch_idx}: images {tuple(images.shape)}, embeddings {tuple(embeddings.shape)}")
    print(f"[DEBUG] losses -> loss_D={loss_D.item():.6f}, loss_G={loss_G.item():.6f}")

    if batch_idx >= 2:  # stop after dry-run
        break

print("[INFO] Dry-run complete. Models ran on GPU successfully, no weights saved.")


[INFO] Using device: cuda
[DEBUG] Loading embeddings from: data/embeddings/faces.npy
[DEBUG] Embeddings loaded: 7136 items, embeddings dim = 512
[WARN] 83 images in 'data/processed/faces' have NO matching embedding and will be skipped (examples): ['1 (1047).jpg', '1 (120).jpg', '1 (1323).jpg', '1 (1519).jpg', '1 (1568).jpg']
[INFO] Using 7136 image-embedding pairs for training.
[INFO] Dataloader ready: 7136 samples, batch_size=4
[INFO] Generator and Discriminator instantiated on GPU.
[INFO] Loading pretrained .pkl from: backend/models/model_weights/stylegan2-ffhq-512x512.pkl
[ERROR] Exception loading .pkl: No module named 'torch_utils'. Continuing without weights.
[INFO] Starting dry-run training on GPU...
